In [0]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

import numpy as np
from PIL import Image

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [4]:
cd ..

/


In [5]:
cd 'gdrive/My Drive/Senior Year/5th: Senior Research Project with Connor/Dataset/finished'

/gdrive/My Drive/Senior Year/5th: Senior Research Project with Connor/Dataset/finished


In [6]:
ls

all.csv                     annotations_flooding.gsheet
annotations_earthquake.csv  annotations_hurricane.csv
annotations_fire.csv        annotations_hurricane.gsheet
annotations_flooding.csv    google_images/


In [0]:
flood_anns = pd.read_csv('annotations_flooding.csv')
hurricane_anns = pd.read_csv('annotations_hurricane.csv')
fire_anns = pd.read_csv('annotations_fire.csv')
earthquake_anns = pd.read_csv('annotations_earthquake.csv')
all_anns = pd.read_csv('all.csv')

In [0]:
flood_anns.dropna(inplace=True)
hurricane_anns.dropna(inplace=True)
fire_anns.dropna(inplace=True)
earthquake_anns.dropna(inplace=True)
all_anns.dropna(inplace=True)

In [0]:
flood_anns.reset_index(inplace=True, drop=True)
hurricane_anns.reset_index(inplace=True, drop=True)
fire_anns.reset_index(inplace=True, drop=True)
earthquake_anns.reset_index(inplace=True, drop=True)
all_anns.reset_index(inplace=True, drop=True)
# img = Image.open('testrgba.png')
# array = np.array(img)
# print(array.shape) 

In [10]:
fire_anns.shape[0]

100

In [11]:
fire_anns.head()

,photo_ID,earthquake,flooding,fire,hurricane,bridge,building damage,lava,roads,utilities,snow,vegetation (low),vegetation (high),river,disaster_type
0,google_images\fire_aerial\0.jpg,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
1,google_images\fire_aerial\1.jpg,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,3.0
2,google_images\fire_aerial\2.jpg,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
3,google_images\fire_aerial\3.jpg,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
4,google_images\fire_aerial\4.jpg,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0


In [12]:
hurricane_anns.head()

,photo_ID,earthquake,flooding,fire,hurricane,bridge,building damage,lava,roads,utilities,snow,vegetation (low),vegetation (high),river,disaster_type
0,google_images/hurricane_aerial_resized/0000.jpg,0,0,0,1,0,1,0,0,0,0,1,0,0,4
1,google_images/hurricane_aerial_resized/0001.jpg,0,0,0,1,0,1,0,0,1,0,1,0,0,4
2,google_images/hurricane_aerial_resized/0002.jpg,0,0,0,1,0,1,0,0,1,0,1,0,0,4
3,google_images/hurricane_aerial_resized/0003.jpg,0,0,0,1,0,0,0,0,0,0,0,1,1,4
4,google_images/hurricane_aerial_resized/0004.jpg,0,0,0,1,0,0,0,0,1,0,0,0,1,4


In [13]:
for i in range(10):
  img = Image.open(hurricane_anns.loc[i, 'photo_ID'])
  img_arr = np.array(img)
  print(img_arr)

[[[ 54  54  54]
  [ 56  56  56]
  [ 47  47  47]
  ...
  [112 112 112]
  [133 133 133]
  [116 116 116]]

 [[ 52  52  52]
  [ 54  54  54]
  [ 49  49  49]
  ...
  [113 113 113]
  [134 134 134]
  [115 115 115]]

 [[ 53  53  53]
  [ 55  55  55]
  [ 54  54  54]
  ...
  [124 124 124]
  [144 144 144]
  [122 122 122]]

 ...

 [[197 197 197]
  [179 179 179]
  [100 100 100]
  ...
  [218 218 218]
  [222 222 222]
  [217 217 217]]

 [[122 122 122]
  [100 100 100]
  [ 62  62  62]
  ...
  [215 215 215]
  [220 220 220]
  [218 218 218]]

 [[ 58  58  58]
  [ 44  44  44]
  [ 43  43  43]
  ...
  [209 209 209]
  [215 215 215]
  [219 219 219]]]
[[[121 121 121]
  [141 141 141]
  [155 155 155]
  ...
  [156 156 156]
  [179 179 179]
  [197 197 197]]

 [[162 162 162]
  [174 174 174]
  [177 177 177]
  ...
  [141 141 141]
  [160 160 160]
  [186 186 186]]

 [[179 179 179]
  [180 180 180]
  [176 176 176]
  ...
  [130 130 130]
  [144 144 144]
  [171 171 171]]

 ...

 [[183 183 183]
  [127 127 127]
  [ 70  70  70]
  ..

In [14]:
cd ..

/gdrive/My Drive/Senior Year/5th: Senior Research Project with Connor/Dataset


In [15]:
ls

 earthquake/   fire_aerial/  'flooding aerial'/   'Hurricane Aftermath'/
 finished/     flooding/     'hurricane aerial'/  'JPG earthquake'/


In [16]:
# loading training images
train_img = []
for img_name in tqdm(all_anns['photo_ID']):
    # defining the image path
    image_path = str(img_name)
    # reading the image
    img = imread(image_path, as_gray=True)
    # print(img) # for testing
    # normalizing the pixel values
    # img /= 255.0   # already between 0 and 1
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_img.append(img)

# converting the list to numpy array
train_x = np.array(train_img)
# defining the target
train_y = all_anns['disaster_type'].values
train_x.shape


# for img_name in tqdm(flood_anns['photo_ID']):
#     # defining the image path
#     image_path = str(img_name) + '.jpg'
#     # reading the image
#     img = imread(image_path, as_gray=True)
#     # normalizing the pixel values
#     img /= 255.0
#     # converting the type of pixel to float 32
#     img = img.astype('float32')
#     # appending the image into the list
#     train_img.append(img)
    
# for img_name in tqdm(hurricane_anns['photo_ID']):
#     # defining the image path
#     image_path = str(img_name) + '.jpg'
#     # reading the image
#     img = imread(image_path, as_gray=True)
#     # normalizing the pixel values
#     img /= 255.0 
#     # converting the type of pixel to float 32
#     img = img.astype('float32')
#     # appending the image into the list
#     train_img.append(img)

# for img_name in tqdm(fire_anns['photo_ID']):
#     # defining the image path
#     image_path = str(img_name) + '.jpg'
#     # reading the image
#     img = imread(image_path, as_gray=True)
#     # normalizing the pixel values
#     img /= 255.0
#     # converting the type of pixel to float 32
#     img = img.astype('float32')
#     # appending the image into the list
#     train_img.append(img)

# for img_name in tqdm(earthquake_anns['photo_ID']):
#     # defining the image path
#     image_path = str(img_name) + '.jpg'
#     # reading the image
#     img = imread(image_path, as_gray=True)
#     # normalizing the pixel values
#     img /= 255.0
#     # converting the type of pixel to float 32
#     img = img.astype('float32')
#     # appending the image into the list
#     train_img.append(img)

# # converting the list to numpy array
# train_x = np.array(train_img)
# # defining the target
# train_y = all_anns['label'].values
# train_x.shape

100%|██████████| 361/361 [03:40<00:00,  1.90it/s]


(361,)